<a href="https://colab.research.google.com/github/kkrusere/youTube-comments-Analyzer/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip -P /tmp/
unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
chmod +x /tmp/chromedriver-linux64/chromedriver
mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver_autoinstaller

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,081 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,357 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/m

In [2]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')


from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_autoinstaller.install()
driver = webdriver.Chrome(options=chrome_options)


from bs4 import BeautifulSoup

In [3]:
# YouTube video URL
video_url = "https://www.youtube.com/watch?v=cZlsZwcIgpc"

In [4]:
try:
    wait = WebDriverWait(driver, 5)

    # Open the YouTube video
    driver.get(video_url)

    wait.until(EC.visibility_of_element_located((By.TAG_NAME, 'body')))

    # Scroll down to load comments
    last_height = 0
    while True:
        # Scroll to the end of the page
        #driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        # Scroll using page-manager element's scroll height
        driver.execute_script("window.scrollTo(0, document.getElementById('page-manager').scrollHeight);")
        time.sleep(3)  # Adjust sleep time as needed

        # Get the current page height
        new_height = driver.execute_script("return document.getElementById('page-manager').scrollHeight")

        # Break the loop if no more content is loaded
        if new_height == last_height:
            break

        last_height = new_height

    # Wait for the comments section to be visible
    comments_section = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "ytd-comments#comments.style-scope.ytd-watch-flexy")))

    # Get the HTML content of the comments section
    comments_html = comments_section.get_attribute('outerHTML')

finally:
    # Close the WebDriver session
    driver.quit()



In [5]:
soup = BeautifulSoup(comments_html, 'html.parser')

# Find the span element with the specified class
comment_count_span = soup.find('span', class_='style-scope yt-formatted-string')

# Extract the text content of the span element
comment_count = comment_count_span.text.strip()

# Print or use the comment count
print("Comment Count:", comment_count)

Comment Count: 23,211


In [6]:
# Find all occurrences of the ytd-comment-thread-renderer element
comment_thread_renderers = soup.find_all('ytd-comment-thread-renderer', class_='style-scope ytd-item-section-renderer')

# Count the number of occurrences
comment_thread_count = len(comment_thread_renderers)

# Print or use the comment thread count
print("Number of ytd-comment-thread-renderer elements:", comment_thread_count)

Number of ytd-comment-thread-renderer elements: 1101


In [ ]:
comments = []
# Iterate through each comment thread renderer
for comment_thread_renderer in comment_thread_renderers:

    # Extracting the comment text
    comment_text_element = comment_thread_renderer.find('yt-attributed-string', id='content-text')
    comment_text = comment_text_element.get_text(strip=True) if comment_text_element else None

    # Extracting the number of likes
    like_count_element = comment_thread_renderer.find('span', class_='style-scope ytd-comment-engagement-bar')
    like_count = like_count_element.get_text(strip=True) if like_count_element else None

    # Extracting the number of replies
    reply_count_element = comment_thread_renderer.find('ytd-button-renderer', id='more-replies')
    reply_count = reply_count_element.get_text(strip=True) if reply_count_element else None

    comments.append(comment_text)

    print("Comment Text:", comment_text)
    print("Like Count:", like_count)
    print("Reply Count:", reply_count)

    print("\n\n\n")


In [4]:
from google.colab import userdata
api_key = userdata.get('YouTubeAPI_key')

In [7]:
import googleapiclient.discovery
import datetime

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=api_key)


In [8]:
def get_video_id(video_url):
  video_id = video_url.split('=')[1]

  return video_id

In [10]:
# Request video details
response = youtube.videos().list(
    part="snippet,statistics",
    id=get_video_id(video_url)
).execute()

# Extract required information
video = response["items"][0]
title = video["snippet"]["title"]
thumbnail_url = video["snippet"]["thumbnails"]["maxres"]["url"]
view_count = video["statistics"]["viewCount"]
like_count = video["statistics"]["likeCount"]
commentCount = video["statistics"]["commentCount"]
date_posted = video["snippet"]["publishedAt"]
description = video["snippet"]["description"]


date_str = date_posted
date_object = datetime.datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")

# Format the date object
date_posted = date_object.strftime("%Y-%m-%d %H:%M:%S")


# Split the description by newlines
paragraphs = description.split("\n")

# Get the first paragraph
first_paragraph = paragraphs[0]

# Generate HTML to display the thumbnail
thumbnail_html = f'<img src="{thumbnail_url}" alt="Thumbnail">'

# Print the information
print("Title:", title)
print("Thumbnail URL:", thumbnail_html)
print("View count:", view_count)
print("Like count:", like_count)
print("Number of Comments:", commentCount)
print("Date posted:", date_posted)
print("Description:", first_paragraph)


Title: Why EVs Are Piling Up At Dealerships In The U.S.
Thumbnail URL: <img src="https://i.ytimg.com/vi/cZlsZwcIgpc/maxresdefault.jpg" alt="Thumbnail">
View count: 4492458
Like count: 36502
Number of Comments: 23211
Date posted: 2023-10-16 16:26:47
Description: In August 2023, it took about twice as long to sell an EV in the U.S. as it did the previous January. Prices of EVs are down 22% year-over-year and that's mainly driven by Tesla. About two thirds of EVs sold are Elon Musk's brand. Companies like Ford have ramped up hybrid production as demand has leveled off. While slightly more than half of consumers say EVs are the future and will eventually replace Internal Combustion Engines, less than a third of dealers say so. This all comes at a time when investments in EVs are more than ever. So what's really going on? Watch the video to learn more.


In [10]:
# Search for videos using the YouTube API
def search_videos(query, youtube = youtube):

  # Request video search
  response = youtube.search().list(
      q=query,
      type="video",
      part="id,snippet",
      maxResults=5
  ).execute()

  # Extract video IDs from search results
  video_ids = []
  for item in response["items"]:
    video_ids.append(item["id"]["videoId"])

  return video_ids

In [14]:
def get_video_info(video_id):
  # Request video details
  response = youtube.videos().list(
      part="snippet,statistics",
      id=video_id
  ).execute()

  # Extract required information
  video = response["items"][0]
  title = video["snippet"]["title"]
  view_count = video["statistics"]["viewCount"]
  like_count = video["statistics"]["likeCount"]
  commentCount = video["statistics"]["commentCount"]
  date_posted = video["snippet"]["publishedAt"]
  description = video["snippet"]["description"]


  date_str = date_posted
  date_object = datetime.datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")

  # Format the date object
  date_posted = date_object.strftime("%Y-%m-%d %H:%M:%S")


  # Split the description by newlines
  paragraphs = description.split("\n")

  # Get the first paragraph
  first_paragraph = paragraphs[0]

  extracted_video_url = f"https://www.youtube.com/watch?v={video_id}"


  # Print the information
  print("Title:", title)
  print("Video URL:", extracted_video_url)
  print("View count:", view_count)
  print("Like count:", like_count)
  print("Number of Comments:", commentCount)
  print("Date posted:", date_posted)
  print("Description:", first_paragraph)
  print("\n\n\n")


In [13]:
search_query = "funny cats"
video_ids = search_videos(search_query)


for video_id in video_ids:
  get_video_info(video_id)

Title: Funny Cat, Dog and Kitten will have you Laugh rolling on the floor 🤣
Video URL: https://www.youtube.com/watch?v=ZDOsB5NcCyM
View count: 74368
Like count: 614
Number of Comments: 24
Date posted: 2024-04-09 13:00:24
Description: Funny Cat, Dog and Kitten will have you Laugh rolling on the floor 🤣 
n



Title: When Your Cat Speaks English Better Than You Think😸 Funny Cat Videos
Video URL: https://www.youtube.com/watch?v=YgaHMWAogbI
View count: 14976
Like count: 443
Number of Comments: 17
Date posted: 2024-04-10 13:00:35
Description: When Your Cat Speaks English Better Than You Think😸 Funny Cat Videos
n



Title: 😹Cats Doing Cat Things😹 (3)
Video URL: https://www.youtube.com/watch?v=3bhkYoMWTFE
View count: 293558584
Like count: 7948971
Number of Comments: 50207
Date posted: 2023-07-26 13:33:50
Description: #cat #cats #catlover #catvideos #catlovers #catshorts #funny #funnyvideo #funnyshorts #funnyvideos #funnycat #cute #cutecat #fyp #fypシ
n



Title: Funny cats 😂 episode 202 #shorts